In [1]:
from math import ceil, sqrt

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(message)s') # %(asctime)s %(levelname)s

In [3]:
C = 10**12
S = 10**8
M = 10**5
F = 0 # TEMP

In [4]:
"""
Calculation of transaction compute mass. For simplicity, the calculation assumes
standard signatures and scripts. However other than that it is accurate.
"""

SPK_STD_LEN = 36 # version len + p2pk len
SIG_STD_LEN = 66 # schnorr sig is 64 bytes + 2 op data bytes
OUT_SIZE = 8 + 2 + 8 + SPK_STD_LEN
IN_SIZE = 32 + 4 + 8 + SIG_STD_LEN + 8
TX_FIELDS_SIZE = 2 + 8 + 8 + 8 + 20 + 8 + 32 + 8

def estimated_tx_size(inputs_num, outputs_num):
    return TX_FIELDS_SIZE + inputs_num * IN_SIZE + outputs_num * OUT_SIZE 

def compute_mass(inputs, outputs):
    inputs_num, outputs_num = len(inputs), len(outputs)
    return 1000 * inputs_num + 10 * SPK_STD_LEN * outputs_num + estimated_tx_size(inputs_num, outputs_num)

In [5]:
def negative_mass(inputs, outputs_num):
    """
    Calculates the negative component of the storage mass formula. Note that there is no dependency on output
    values but only on their count. The code is designed to maximize precision and to avoid intermediate overflows.
    """
    inputs_num = len(inputs)
    if outputs_num == 1 or inputs_num == 1 or (outputs_num == 2 and inputs_num == 2):
        return sum(C//v for v in inputs)
    return inputs_num*(C//(sum(inputs)//inputs_num)) 

def storage_mass(inputs, outputs):
    N = negative_mass(inputs, outputs_num=len(outputs))
    P = sum(C//o for o in outputs)
    return max(P-N, 0)

def mass(inputs, outputs):
    return max(storage_mass(inputs, outputs), compute_mass(inputs, outputs)) 

In [6]:
def build_txs(inputs, payment):
    txs = []
    while storage_mass(inputs, outputs=[payment, sum(inputs) - payment - F]) > M:
        T = sum(inputs) - F
        N = negative_mass(inputs, 2) 
        D = (M + N)*T/C
        alpha = (D - sqrt(D**2 - 4*D))/(2*D)              # single step optimization, taking the smaller solution 
        outputs = [ceil(alpha * T), T - ceil(alpha * T)]  # round *up* in order to not increase the mass above M
        txs.append((inputs, outputs))
        logger.debug(storage_mass(inputs, outputs))
        inputs = outputs
    txs.append((inputs, [payment, sum(inputs) - payment - F]))
    logger.debug(storage_mass(*txs[-1]))
    return txs

In [7]:
mass([S], [S//2, S//2]), mass([10000*S], [10000*S//2, 10000*S//2])

(30000, 2040)

In [8]:
logger.setLevel(logging.DEBUG)
txs = build_txs([S], int(S*0.009))
len(txs)

99999
99999
99999
99999
99999
99999
99998
99998
99998
99999
99998
11217


12